In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
import shutil

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import zipfile
zip_path = "/content/drive/MyDrive/computer visison"

for root, dirs, files in os.walk(zip_path):
    print(f"\nDirectory: {root}")
    for file in files:
        print(f" - {file}")

Streaming output truncated to the last 5000 lines.
 - 00023_00003_00010.png
 - 00023_00008_00002.png
 - 00023_00004_00004.png
 - 00023_00004_00029.png
 - 00023_00003_00017.png
 - 00023_00009_00013.png
 - 00023_00006_00001.png
 - 00023_00007_00027.png
 - 00023_00012_00008.png
 - 00023_00009_00015.png
 - 00023_00008_00019.png
 - 00023_00015_00009.png
 - 00023_00006_00003.png
 - 00023_00015_00011.png
 - 00023_00009_00018.png
 - 00023_00014_00018.png
 - 00023_00016_00014.png
 - 00023_00011_00020.png
 - 00023_00007_00029.png
 - 00023_00004_00023.png
 - 00023_00001_00029.png
 - 00023_00002_00018.png

Directory: /content/drive/MyDrive/computer visison/archive/Train/19
 - 00019_00000_00029.png
 - 00019_00000_00017.png
 - 00019_00000_00026.png
 - 00019_00000_00019.png
 - 00019_00000_00028.png
 - 00019_00000_00007.png
 - 00019_00000_00024.png
 - 00019_00000_00023.png
 - 00019_00000_00022.png
 - 00019_00000_00015.png
 - 00019_00000_00010.png
 - 00019_00000_00000.png
 - 00019_00000_00016.png
 - 00

In [7]:
!pip install torch torchvision torchaudio

In [8]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [9]:
# Parameters
img_height, img_width = 128, 128  # Image dimensions for resizing
batch_size = 32  # Batch size for training/testing


In [11]:
train_dir = "/content/d"


In [12]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

''
 Step 1: Set file paths for training and testing datasets.
Each of these folders should contain subfolders named after the classes,
with images inside each class-specific subfolder.
'''

'''
 Step 2: Define image size and batch size.
Images will be resized to 128x128 pixels before feeding to the model.
Batch size controls how many images are processed at once.
'''
'''
Step 3: Set up data augmentation for the training dataset.
Augmentation increases data variety and helps the model generalize better.
Common transformations include rotation, zoom, shifting, flipping, etc.
'''

'''
Step 4: Set up a test data generator.
Only rescaling is applied (no augmentation) to keep test data consistent.
'''

'''
Load training images with the defined augmentations.
Images will be resized, augmented, and loaded in batches.
Each subfolder in 'train_dir' is treated as a class label.
'''

'''
 Step 6: Load testing images with only rescaling.
This helps evaluate model performance on clean, unmodified data.
''

In [13]:
# Paths for train and test datasets
train_dir = '/content/drive/MyDrive/computer visison/archive/Train'
test_dir = '/content/drive/MyDrive/computer visison/archive/Test'

# Parameters
img_height, img_width = 128, 128  # Image dimensions for resizing
batch_size = 32  # Batch size for training/testing



# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Zoom
    horizontal_flip=True,  # Horizontal flip
    fill_mode='nearest'  # Fill mode for missing pixels
)

# Only rescaling for the testing dataset
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create testing data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 8146 images belonging to 19 classes.
Found 0 images belonging to 0 classes.


The `load_data` function loads the GTSRB dataset by traversing class subdirectories, resizing each image to the specified dimensions, and storing them as NumPy arrays. It returns the images, their corresponding class labels, and the list of class names, ready for use in model training.



In [14]:

def load_data(data_dir, img_height, img_width):
    images = []
    labels = []
    class_names = os.listdir(data_dir)
    class_names.sort()

    for class_id, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize((img_width, img_height))
                images.append(np.array(img))
                labels.append(class_id)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names


In [15]:
num_classes = 43

In [16]:
!pip install pillow

In [17]:
from PIL import Image

In [18]:
from sklearn.model_selection import train_test_split


In [19]:
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

This code defines a function load_data that loads and preprocesses the GTSRB dataset, processes images, and prepares them for model training. Here's a breakdown of what the code does:

Loading Data:

The function load_data takes the dataset directory, image height, and width as parameters.

It reads all subdirectories (representing classes) in the dataset directory and prints the class names found.

For each image file in each class subdirectory, it reads the image using OpenCV (cv2.imread), resizes it to the specified dimensions, and appends it to the images list.

Corresponding class labels are stored in the labels list.

Configuration and Setup:

img_height, img_width, and batch_size are set for image preprocessing and training.

A check is performed to ensure the training directory exists. If it doesn't, a FileNotFoundError is raised.

Data Preprocessing:

The function load_data is called to load and preprocess the images into X and y arrays, with class names returned.

If no images are found, a ValueError is raised.

The image data (X) is normalized by dividing by 255.0 to scale pixel values to the range [0, 1].

The labels (y) are one-hot encoded to prepare for classification.

Splitting Data:

The data is split into training and validation sets using train_test_split, with 20% of the data used for validation.

Data Augmentation:

A data augmentation generator (train_datagen) is created using ImageDataGenerator to apply random transformations (e.g., rotations, zoom, shifts) to the training data during training to help improve model generalization

In [20]:

def load_data(data_dir, img_height, img_width):
    images = []
    labels = []
    class_names = os.listdir(data_dir)
    class_names.sort()

    print(f"Classes found: {class_names}")

    for label, class_name in enumerate(class_names):
        class_path = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        for img_file in tqdm(os.listdir(class_path), desc=f"Loading {class_name}"):
            img_path = os.path.join(class_path, img_file)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.resize(img, (img_width, img_height))
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Failed to process {img_path}: {e}")

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

# ==== Configuration ====
img_height, img_width = 128, 128
batch_size = 32
num_classes = 19

# Path for training dataset
train_dir = '/content/drive/MyDrive/computer visison/archive/Train'

# Check if path exists
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Train directory does not exist: {train_dir}")

# ==== Load Data ====
X, y, class_names = load_data(train_dir, img_height, img_width)

# Check if data is loaded
if X.size == 0 or y.size == 0:
    raise ValueError("No training data found. Please check the dataset directory.")

print(f"Total samples: {X.shape[0]}")
print(f"Number of classes: {len(class_names)}")

# Normalize pixel values to [0, 1]
X = X / 255.0

# One-hot encode labels
y = to_categorical(y, 19)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False
)


Classes found: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25']


Loading 0: 0it [00:00, ?it/s]
Loading 1: 0it [00:00, ?it/s]
Loading 10: 0it [00:00, ?it/s]
Loading 11: 0it [00:00, ?it/s]
Loading 12: 0it [00:00, ?it/s]
Loading 13: 0it [00:00, ?it/s]
Loading 14: 0it [00:00, ?it/s]
Loading 15: 0it [00:00, ?it/s]
Loading 25: 100%|██████████| 1501/1501 [00:16<00:00, 90.89it/s] 


Total samples: 8146
Number of classes: 19
Training set size: 6516
Validation set size: 1630


In [21]:
# ==== CNN Model Definition ====
def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [23]:

# Clear Keras cache
shutil.rmtree(os.path.expanduser('~/.keras'), ignore_errors=True)

# Clear TensorFlow cache (optional, but might help)
tf.keras.backend.clear_session()

In [25]:
# Initialize the model
input_shape = (img_height, img_width, 3)
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Before training the model, specify the device (CPU or GPU)
with tf.device('/CPU:0'):  # Force execution on CPU
    # Train the model
    history = model.fit(
        train_datagen.flow(X_train, y_train, batch_size=batch_size),
        validation_data=(X_val, y_val),
        epochs=20,
        steps_per_epoch=X_train.shape[0] // batch_size,
        validation_steps=X_val.shape[0] // batch_size
    )

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packa

In [ ]:
# Visualize a few test predictions
num_samples = 5
for i in range(num_samples):
    plt.imshow(x_test[i])
    plt.title(f"True: {y_true[i]}, Predicted: {y_pred[i]}")
    plt.axis("off")
    plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average='weighted'))
print("Recall:", recall_score(y_true, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_true, y_pred, average='weighted'))

# Classification report
print("\nClassification Report:\n", classification_report(y_true, y_pred))

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# Plot Training and Validation Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Plot Training and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()